In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('RAW_recipes.csv')
df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [2]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


,id,minutes,contributor_id,n_steps,n_ingredients
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000


In [3]:
df.isna().sum()

name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64

In [4]:
df = df[~df['name'].isnull()]
df.isna().sum()

name                 0
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64

In [5]:
df['tags'] = df['tags'].apply(ast.literal_eval)
tags = df['tags'].explode()
tags

0         60-minutes-or-less
0               time-to-make
0                     course
0            main-ingredient
0                    cuisine
                 ...        
231636               dietary
231636          comfort-food
231636            taste-mood
231636                 sweet
231636    number-of-servings
Name: tags, Length: 4141675, dtype: object

In [6]:
len(tags.unique())

552

In [7]:
tags_counter = tags.value_counts()
tags_filtered = tags_counter[tags_counter>10000]
tags_filtered

tags
preparation        230545
time-to-make       225325
course             218147
main-ingredient    170446
dietary            165090
                    ...  
savory              10544
sweet               10543
potatoes            10293
summer              10039
vegan               10012
Name: count, Length: 82, dtype: int64

In [8]:
df['nutrition'] = df['nutrition'].apply(ast.literal_eval)
df[['calories', 'fat', 'sugar', 'sodium', 'protein', 'saturated_fat', ' carbohydrates']] = df['nutrition'].apply(pd.Series)
df = df.drop(columns = ['nutrition'])
df.head()
#Could use the machine learning model to predict micro/macro nutrient profile for each ingredient and replace this with that

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,calories,fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [9]:
df['steps'] = df['steps'].apply(ast.literal_eval)
df['steps']
#Could do NLP stuff here

0         [make a choice and proceed with recipe, depend...
1         [preheat oven to 425 degrees f, press dough in...
2         [brown ground beef in large pot, add chopped o...
3         [place potatoes in a large pot of lightly salt...
4         [mix all ingredients& boil for 2 1 / 2 hours ,...
                                ...                        
231632    [heat oil in a 4-quart dutch oven, add celery ...
231633            [mix all ingredients together thoroughly]
231634    [in a bowl , combine the mashed yolks and mayo...
231635    [place melted butter in a large mixing bowl an...
231636    [whip sugar and shortening in a large bowl , a...
Name: steps, Length: 231636, dtype: object

In [10]:
df['ingredients'] = df['ingredients'].apply(ast.literal_eval)
df['ingredients']
#NLP stuff here too

0         [winter squash, mexican seasoning, mixed spice...
1         [prepared pizza crust, sausage patty, eggs, mi...
2         [ground beef, yellow onions, diced tomatoes, t...
3         [spreadable cheese with garlic and herbs, new ...
4         [tomato juice, apple cider vinegar, sugar, sal...
                                ...                        
231632    [celery, onion, green sweet pepper, garlic clo...
231633    [paprika, salt, garlic powder, onion powder, d...
231634    [hard-cooked eggs, mayonnaise, dijon mustard, ...
231635    [butter, eagle brand condensed milk, light bro...
231636    [granulated sugar, shortening, eggs, flour, cr...
Name: ingredients, Length: 231636, dtype: object

In [11]:
ingredients = df['ingredients'].explode()
ingredients_counter = ingredients.value_counts()
ingredients_counter

ingredients
salt                                          85746
butter                                        54975
sugar                                         44535
onion                                         39065
water                                         34914
                                              ...  
szechuan chile flakes                             1
herb oil                                          1
kosher pickles                                    1
cholesterol-free margarine                        1
birds eye frozen japanese-style vegetables        1
Name: count, Length: 14942, dtype: int64

In [12]:
#THIS IS JUST TO MAKE MINIMAL VIABLE PRODUCT
#WILL REMOVE LATER
ingredients_filtered = ingredients_counter[ingredients_counter>2000]
ingredients_filtered

ingredients
salt                85746
butter              54975
sugar               44535
onion               39065
water               34914
                    ...  
warm water           2054
almond extract       2046
swiss cheese         2043
ground coriander     2041
mushroom             2020
Name: count, Length: 176, dtype: int64

In [13]:
#Since ingredients are similar could standarize with NLP
def value_checker(row,values):
    return pd.Series({value: value in row for value in values})

In [14]:
#Going to create boolean columns for each ingredient
ingredients_columns = df['ingredients'].apply(value_checker, values = list(ingredients_filtered.index))
ingredients_columns

,salt,butter,sugar,onion,water,eggs,olive oil,flour,milk,garlic cloves,...,cooking spray,beef broth,fresh mushrooms,shallot,"lemon, juice of",warm water,almond extract,swiss cheese,ground coriander,mushroom
0,True,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,False,False,True,True,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
231633,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231634,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231635,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
tags_columns = df['tags'].apply(value_checker, values = list(tags_filtered.index))
tags_columns

,preparation,time-to-make,course,main-ingredient,dietary,easy,occasion,cuisine,low-in-something,main-dish,...,technique,high-in-something,soups-stews,beverages,potluck,savory,sweet,potatoes,summer,vegan
0,True,True,True,True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
2,True,True,True,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,True,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,True,True,True,True,True,False,True,True,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,True,True,True,True,False,True,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
231633,True,True,True,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
231634,True,True,True,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231635,True,True,True,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [16]:
tags_ingredients_columns = pd.concat([ingredients_columns, tags_columns], axis = 1)
tags_ingredients_columns.head()

,salt,butter,sugar,onion,water,eggs,olive oil,flour,milk,garlic cloves,...,technique,high-in-something,soups-stews,beverages,potluck,savory,sweet,potatoes,summer,vegan
0,True,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,True,False,True,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [17]:
#Scraping some columns that seem useless
df_features = df.drop(columns= ['contributor_id', 'submitted', 'tags', 'steps', 'description', 'ingredients', 'name', 'n_steps', 'n_ingredients', 'minutes'])
df_features = pd.concat([df_features, tags_ingredients_columns], axis = 1)
df_features.head()

,id,calories,fat,sugar,sodium,protein,saturated_fat,carbohydrates,salt,butter,...,technique,high-in-something,soups-stews,beverages,potluck,savory,sweet,potatoes,summer,vegan
0,137739,51.5,0.0,13.0,0.0,2.0,0.0,4.0,True,True,...,False,False,False,False,False,False,False,False,False,False
1,31490,173.4,18.0,0.0,17.0,22.0,35.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
2,112140,269.8,22.0,32.0,48.0,39.0,27.0,5.0,True,False,...,False,False,False,False,False,False,False,False,False,False
3,59389,368.1,17.0,10.0,2.0,14.0,8.0,20.0,True,False,...,False,False,False,False,False,False,False,True,False,False
4,44061,352.9,1.0,337.0,23.0,3.0,0.0,28.0,True,False,...,True,False,False,False,False,False,False,False,False,False


In [18]:
#Since some of the tags are ingredients, its useless and now we can find it with the new df
dup_cols = df_features.columns[df_features.columns.duplicated()].to_list()
dup_cols
df_features_cleaned = df_features.loc[:, ~df_features.columns.duplicated()]
df_features_cleaned.columns.value_counts()

vegan        1
id           1
calories     1
fat          1
sugar        1
            ..
butter       1
onion        1
water        1
eggs         1
olive oil    1
Name: count, Length: 262, dtype: int64

In [19]:
#Looking online I found that parquet files are supposed to be better with large datasets versus csv, could ask Kevin thoughts
df_features_cleaned.to_parquet('recipe_features.parquet')

In [20]:
#To upload to github, making a tag/ingredient dataframe, for some reason could not drop carbs???
df_ingredients_tags = df.drop(columns = ['name', 'steps', 'description', 'contributor_id', 'submitted', 'n_ingredients', 
                                    'n_steps', 'minutes', 'calories', 'fat', 'sugar', 'sodium', 'protein',
                                    'saturated_fat'])
df_ingredients_tags.head()

,id,tags,ingredients,carbohydrates
0,137739,"[60-minutes-or-less, time-to-make, course, mai...","[winter squash, mexican seasoning, mixed spice...",4.0
1,31490,"[30-minutes-or-less, time-to-make, course, mai...","[prepared pizza crust, sausage patty, eggs, mi...",1.0
2,112140,"[time-to-make, course, preparation, main-dish,...","[ground beef, yellow onions, diced tomatoes, t...",5.0
3,59389,"[60-minutes-or-less, time-to-make, course, mai...","[spreadable cheese with garlic and herbs, new ...",20.0
4,44061,"[weeknight, time-to-make, course, main-ingredi...","[tomato juice, apple cider vinegar, sugar, sal...",28.0


In [21]:
df_ingredients_tags.to_parquet('recipes_ingredients_tags.parquet')

In [22]:
# Create a dataframe with steps and description, once again I can't get rid of carbs???????????????????????
df_steps = df.drop(columns=['contributor_id', 'submitted', 'n_ingredients', 'n_steps', 'minutes', 'calories', 'fat',
                                      'sugar', 'sodium', 'protein', 'saturated_fat', 'tags'])
df_steps.head()

,name,id,steps,description,ingredients,carbohydrates
0,arriba baked winter squash mexican style,137739,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",4.0
1,a bit different breakfast pizza,31490,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",1.0
2,all in the kitchen chili,112140,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",5.0
3,alouette potatoes,59389,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",20.0
4,amish tomato ketchup for canning,44061,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",28.0


In [23]:
df_steps.to_parquet('recipe_steps.parquet')

STREAMLIT 

In [24]:
df_test = df_ingredients_tags.copy()
df_test.head()

,id,tags,ingredients,carbohydrates
0,137739,"[60-minutes-or-less, time-to-make, course, mai...","[winter squash, mexican seasoning, mixed spice...",4.0
1,31490,"[30-minutes-or-less, time-to-make, course, mai...","[prepared pizza crust, sausage patty, eggs, mi...",1.0
2,112140,"[time-to-make, course, preparation, main-dish,...","[ground beef, yellow onions, diced tomatoes, t...",5.0
3,59389,"[60-minutes-or-less, time-to-make, course, mai...","[spreadable cheese with garlic and herbs, new ...",20.0
4,44061,"[weeknight, time-to-make, course, main-ingredi...","[tomato juice, apple cider vinegar, sugar, sal...",28.0


In [25]:
#Func to check if tags are in the row
def present_tags(item_tags, selected):
    return all(string in item_tags for string in selected)

In [26]:
df_test['tags_bool'] = df_test['tags'].apply(present_tags, selected = ['poultry', '30-minutes-or-less'])
df_test = df_test[df_test['tags_bool']== True]
df_test.head()

,id,tags,ingredients,carbohydrates,tags_bool
57,32169,"[30-minutes-or-less, time-to-make, course, mai...","[tomatoes, garlic, onion, button mushrooms, ho...",6.0,True
75,42570,"[30-minutes-or-less, time-to-make, course, mai...","[boneless chicken breast, garlic, salt, cumin,...",0.0,True
233,219681,"[30-minutes-or-less, time-to-make, course, mai...","[chicken breasts, olive oil, lemon juice, whit...",8.0,True
336,399278,"[30-minutes-or-less, time-to-make, course, mai...","[cayenne pepper sauce, vegetable oil, granulat...",1.0,True
447,371549,"[30-minutes-or-less, time-to-make, course, mai...","[jalapeno peppers, fresh cilantro, light beer,...",5.0,True


In [30]:
#Func for ingredients in row
ingredients_selected = ['chicken', 'lentils']
def ingredients_row(row):
    ingredients_matched = row
    ingredients_str = ' '.join(str(ingredients).lower() for ingredients in ingredients_matched)
    
    for item in ingredients_selected:
        item = item.strip('s')
        if item not in ingredients_str:
            return False
    return True


In [31]:
df_test['ingredients_match'] = df_test['ingredients'].apply(ingredients_row)
df_test[df_test['ingredients_match'] == True]

,id,tags,ingredients,carbohydrates,tags_bool,ingredients_match
16018,204669,"[30-minutes-or-less, time-to-make, course, mai...","[skinless chicken breasts, split peas, red len...",8.0,True,True
46564,66735,"[30-minutes-or-less, time-to-make, course, mai...","[olive oil, onion, garlic, red lentils, chicke...",21.0,True,True
123030,287439,"[30-minutes-or-less, time-to-make, course, mai...","[boneless skinless chicken breasts, water, tom...",15.0,True,True
180614,363669,"[lactose, 30-minutes-or-less, time-to-make, co...","[whole chicken, olive oil, onion, garlic clove...",29.0,True,True


In [32]:
df_steps.head()

,name,id,steps,description,ingredients,carbohydrates
0,arriba baked winter squash mexican style,137739,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",4.0
1,a bit different breakfast pizza,31490,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",1.0
2,all in the kitchen chili,112140,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",5.0
3,alouette potatoes,59389,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",20.0
4,amish tomato ketchup for canning,44061,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",28.0


In [ ]:
#Since all food.com recipe urls follow the same format, can use this to give the link to the actual recipe
rec = df_steps[df_steps['id'] == 66735]
rec['steps'].values[0], rec['name'].values[0]
name = rec['name'].values[0].replace(' ', '-')
link = f"https://www.food.com/recipe/{name}-{rec['id'].values[0]}"
link

'https://www.food.com/recipe/chicken-with-red-lentils-66735'

In [39]:
#This is a way to recommend similar recipes if the user likes a certain recipe

rec_features = df_features_cleaned[df_features_cleaned['id'] == 66735]
rec_features = rec_features.drop(columns=['id']).values.reshape(1,-1)

#cosine similarity
cosine_sim = cosine_similarity(rec_features, df_features_cleaned.drop(columns=['id']))
sim_scores = list(zip(df_features_cleaned['id'].values, cosine_sim[0]))
sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse= True)
sim_scores = sim_scores[1:6]
rec_indices = [i[0] for i in sim_scores]

recs = df_steps[df_steps['id'].isin(rec_indices)][['id', 'name', 'description']]
recs

,id,name,description
42858,13318,chicken and pasta,NaN
44912,292894,chicken noodle goulash abs diet,my husband does the abs diet. this is one of t...
107617,359825,honey mustard cracker crumb chicken,this is a healthier version of fried chicken. ...
150669,161093,oven baked fish and chips,"from family circle, uk. a healthy version of ..."
168905,458924,quick double tomato chicken with artichoke hea...,"entered for safe-keeping, this is almost a ""cu..."
